In [1]:
import numpy
import pandas

import gensim
from gensim import corpora

from pythainlp.tokenize import word_tokenize

import re

In [18]:
def clean(doc):
    while True:
        new_doc = re.sub('[^\u0E00-\u0E7F]+', '', doc)
        if doc == new_doc:
            break
        else:
            doc = new_doc

    return doc.strip()

def load_corpus(file_directory):
    comments = []
    labels = []
    
    with open(file_directory, 'r') as f:
        start = False
        num = -2

        for line in f:
            if line.startswith('comment'):
                num += 1
                if num < 0:
                    continue

                comment = ''.join(x for x in line.split(':')[2:])
                comments.insert(num, comment)
                labels.insert(num, line.split(':')[1])
                start = True

            elif start:
                comments[num] += line
        
    return comments, labels
                
def get_tokenized_corpus(corpus):
    tokenized_corpus = []
    
    for num in range(len(corpus)):
        tokenized_corpus.insert(num, clean(corpus[num]))
        tokenized_corpus[num] = word_tokenize(tokenized_corpus[num], engine='icu')
        
    return tokenized_corpus

### Preprocess Corpus

In [19]:
def remove_stop_words (len_corpus, dictionary, appear_rate):
    dictionary.filter_extremes(no_above=len_corpus * appear_rate)
    return dictionary

In [20]:
corpus, labels = load_corpus('../data/facebook/1/ผู้บริโภค - AllCafe.txt')
tokenized_corpus = get_tokenized_corpus(corpus)

len_corpus = len(corpus)
clusters = list(set(labels))
print('Total documents', len_corpus)

dictionary = corpora.Dictionary(tokenized_corpus)
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

dictionary = remove_stop_words(len_corpus, dictionary, 0.9)
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

Total documents 1091


In [14]:
for doc in idx_corpus:
    print('[', end='')
    for id in doc:
        if id != -1:
            print('(' + dictionary[id] + '),', end=' ')
    print(']')

[(ลด), (ความ), (ดีด), (ของ), (กาแฟ), (ลง), (หน่อย), (ครับ), (กาแฟ), (กด), (แก้ว), (บาท), (นะ), (ครับ), (ช่วย), (ลด), (ความ), (ดีด), (ลง), (หน่อย), (นะ), (ครับ), (แค่), (โดน), (ไป), (ครึ่ง), (แก้ว), (กู), (จะ), ]
[(คน), (ที่), (บอก), (ว่า), (พนักงาน), (ทำ), (หน้า), (เหมือน), (ส้นตีน), (ว่า), (คุณ), (ไม่ใช่), (ของ), (เขา), (ครับ), ]
[(สงสาร), (พนัก), (งาน), (เซ), (เว่), (บ), (เงิน), (เดือน), (ไม่), (มาก), (หน้าที่), (เกิน), (เงิน), (เดือน), (ครับ), (ขาย), (ของ), (ขาย), (กา), (เติม), (เงิน), (ล่าสุด), (จะ), (รับ), (ฝาก), (เงิน), (ให้), ]
[(ชง), (ให้), (มัน), (หน่อย), (บางที), (กู), (รีบ), (ไป), (ทำงาน), (มัว), (แต่), (คุย), (กัน), (อยู่), (ได้), (ชิ), (บ), (หาย), ]
[(ไม่), (ฝาก), (ไร), (มาก), (ค่ะ), (สงสาร), (พนักงาน), (มึง), (จะ), (ให้), (เค้า), (เป็น), (ทุก), (อย่าง), (เลย), (หรอ), ]
[(เอา), (ผัก), (พวก), (และ), (ซอส), (ลับ), (มา), (ชอบ), (แบบ), (ก่อน), (เขา), (ถึง), (กาแฟ), (ลด), (ความ), (หวาน), (ลง), (หน่อย), (แค่), (เบา), (หวาน), (ก็), (จะ), (ขึ้น), (ละ), ]
[(คน), (ที่), (บอก), (พนัก

[(ไม่), (เข้ม), (ไม่), (ดีด), (สั่ง), (เย็น), (แก้ว), (นิด), (เดียว), (กู), (ต้อง), (ไป), (หา), (แดก), (ร้าน), (ข้าง), (หน้า), (อีก), (แก้ว), ]
[(เอา), (แบบ), (ว่า), (เวลา), (รีบ), (ช่วย), (มา), (คิด), (เงิน), (หน่อย), (ได้), (มั้ย), (คือ), (มี), (เครื่อง), (เครื่อง), (ทำไม), (สุดท้าย), (เห็น), (คิด), (เงิน), (แค่), (คน), (เดียว), ]
[(ชอบ), (พนักงาน), (เซ), (ที่), (เป็น), (คะ), (เพราะ), (เวลา), (พูด), (ครับ), (เป็น), (ลูกค้า), (เท), (จะ), (คำ), (ว่า), (คะ), (ชอบ), (ที่), (มี), (อย่าง), (พวก), (คะ), (เรื่อง), (กาแฟ), (แต่), (อย่าง), (เอา), (ะ), ]
[(กู), (ไม่รู้), (กู), (กด), (เอา), (อย่าง), (เดียว), (น้ำ), (แข็ง), (ไม่), (ต้อง), (เยอะ), (กู), (เอง), (ได้), (แก้ว), (ใหญ่), (นี่), (ดีด), (ขับ), (รถ), (ได้), (เดียว), (กว่า), (จะ), (หลับ), (ได้), (เกือบ), (แม่), (ง), (โคตร), ]
[(เครื่อง), (ออ), (มัน), (น่า), (จะ), (ชง), (ได้), (กาแฟ), (คุณ), (กัน), (ทุก), (ที่), (มั้ย), (บางที), (กาแฟ), (มี), (กลิ่น), (ไหม้), (บาง), (ที่), (ไม่), (เป็น), (ความ), (หวาน), (ถึง), (จะ), (ตาม), (สูตร), (แต่), (พ

[(เอา), (ชา), (มะนาว), (ตู้), (กด), (มา), (จำ), (สาขา), (หน่อย), (และ), (ชง), (เติม), (ด้วย), (ไป), (ที), (ไร), (กด), (ไม่), (พอ), (แก้ว), (ซะ), (ที), (บาง), (สาขา), (ขาย), (ไป), (เลย), (แต่), (กาแฟ), (มี), (สูตร), (แบ่ง), (ที่), (ให้), (น้ำ), (กด), (อื่น), (บ้าง), (จะ), (ดี), (ใจ), (สุดๆ), (เลย), (ค่ะ), ]
[(โคตร), (ชอบ), (ครีม), (ของ), (ร้าน), (นี้), (เลย), (อร่อย), (มา), (กก), (กก), (แต่), (ให้), (น้อย), (อ่ะ), (แล้ว), (แต่), (พนักงาน), (ด้วย), (ราคา), (น่า), (รสชาติ), (ดี), (ชอบ), (นะ), (แต่), (บางที), (ก็), (สั่ง), (นาน), (กว่า), (จะ), (ได้), ]
[(คน), (ชง), (หน้า), (ไม่), (เคย), (บาง), (คน), (ชง), (พอ), (กิน), (ได้), (บาง), (คน), (ชง), (ยัง), (กะ), (น้ำ), (ล้าง), (ตีน), (จะ), (หวาน), (ไป), (ไหน), (แค่), (นี้), (เบา), (หวาน), (กู), (ก้อ), (กว่า), (ละ), (จะ), (ให้), (เบา), (หวาน), (กู), (เลย), (รึ), (ไง), (วะ), ]
[(ปรับ), (ปรุง), (พนักงาน), (พนัก), (งาน), (น่ะ), (ให้), (เค้า), (ดีๆ), (หน่อย), (ใช้), (คุ้ม), (มาก), (พนักงาน), (เซ), (หรือ), (เห็น), (แล้ว), (สงสาร), ]
[(พอ), (มี), (พนัก

[(อยาก), (ให้), (พนัก), (งาน), (เซ), (เว่น), (คุย), (กัน), (ใน), (เวลา), (งาน), (ให้), (น้อย), (ลง), (เพราะ), (บางที), (กู), (รีบ), (รอ), (จ่าย), (เงิน), (แต่ละ), (ที), (เด็ก), (แม่), (คุย), (ใส่), (กัน), (อยู่), (นั่น), (แหละ), (คือ), (กู), (รีบ), (ไง), (และ), (กู), (ก็), (เชื่อ), (ว่า), (ลูกค้า), (คน), (เค้า), (ก็), (รีบ), (หรือ), (ไม่), (ก็), (กะ), (ควร), (จะ), (มี), (พนักงาน), (สัก), (คน), (เพื่อ), (ความ), (นะ), (ครับ), ]
[(ลด), (ความ), (หวาน), (ลง), (หน่อย), (คือ), (บางที), (กุ), (ก้อ), (กด), (น้ำ), (แข็ง), (น้อย), (เพราะ), (เย็น), (อยู่), (แล้ว), (แต่), (แมร่), (ง), (หวาน), (จน), (ต้อง), (เติม), (น้ำ), (เปล่า), (ลง), (ไป), (ผสม), (แดก), (คอ), (ชิ), (บ), (หาย), ]
[(ชา), (อร่อย), (ยยย), (ปรับปรุง), (เรื่อง), (เดียว), (คือ), (มี), (ตลอด), (ชม), (ได้), (ไหม), (แบบ), (เวลา), (ตี), (อยาก), (กิน), (ชา), (นม), (อะไร), (อย่า), ]
[(พนักงาน), (ไม่), (มา), (เมื่อ), (คืน), (ผัว), (ไม่), (หน้า), (เป็น), (หัว), (แม่), (อย่า), (เอา), (มัน), (มา), (หน้า), (เคา), (เตอร์), ]
[(ชอบ), (โก), (แลต), (อ

[(ปรับปรุง), (โดย), (การ), (เอา), (แม่), (งอ), (อก), (ไป), (เหอะ), (เก็บ), (เยอะ), (ทำ), (ไม่), (ได้), (ก้), (โดน), (ด่า), (ชง), (ไม่), (อร่อย), (โดน), (คอม), (เพ), (ออ), (อ), (เอา), (มัน), (ออก), (ไป), (ส), (ติ), ]
[(บาง), (สาขา), (สั่ง), (ไม่), (หวาน), (แต่), (ทำไม), (เป็น), (จืด), (เหมือน), (น้ำ), (เปล่า), (เลย), (ปกติ), (ถ้า), (ไม่), (หวาน), (หรือ), (หวาน), (น้อย), (มัน), (ก็), (จะ), (ต้อง), (ยิ่ง), (เข้ม), (ยิ่ง), (ขม), (รส), (กาแฟ), (สิ), (ะ), (แต่), (บาง), (สาขา), (อร่อย), (มาก), ]
[(เวฟ), (ไม่), (งง), (มา), (กก), (กก), (เย็น), (มาก), (กก), (กก), (กก), (กก), (กก), (กก), (กก), (กก), (ก), (ก็), ]
[(เลิก), (ให้), (ซื้อ), (ของ), (หน้า), (เคา), (เตอร์), (เถอะ), (เพราะ), (เวลา), (เรา), (ไม่), (ซื้อ), (บาง), (ครั้ง), (พนักงาน), (มัน), (จะ), (ออก), ]
[(ไม่), (บอก), (เอา), (หวาน), (น้อย), (เลย), (ค่ะ), (คือ), (แม่), (ง), (ไม่), (ใส่), (น้ำตาล), (เลย), (แต่), (ถ้า), (ไม่), (บอก), (แม่), (หวาน), (มา), (เลย), (เบา), (หวาน), (ถาม), (หา), (เลย), (ส่วน), (ราคา), (ต้อง), (ได้), ]
[(อยาก), (ให้)

[(รสชาติ), (ค่ะ), (เพราะ), (ปกติ), (ไม่), (กิน), (กาแฟ), (ขม), (สำหรับ), (เรา), (เรา), (ว่า), (อร่อย), (ดี), (และ), (ราคา), (ก็), (ไม่), (แพง), (ถ้า), (ราคา), (ทุก), (ใน), (เท่า), (กัน), (หมด), (ก็), (จะ), (เลือก), (ของ), (นี่), (แหละ), ]
[(กิน), (แต่), (อเมริกา), (โน่), (เย็น), (ไม่), (ใส่), (น้ำตาล), (ดี), (ไม่), (หอม), (มาก), (แต่), (รสชาติ), (เหมือน), (กัน), (เป็น), (ส่วน), (ใหญ่), (ราคา), (เท่า), (นี้), (ก็), (โอ), (อยู่), (ชอบ), (ไม่), ]
[(อยาก), (ให้), (มี), (ชา), (เขียว), (แก้ว), (ใหญ่), (งอ), (ร่อย), (ชอบ), (หอม), (แก้ว), (เล็ก), (ไม่), (ค่ะ), ]
[(ควร), (เอา), (ออก), (ไป), (จาก), (เซ), (เว่), (สา), (ยัง), (จะ), (เอา), (มา), (ไว้), (อีก), (ไม่), (รุ้), (คน), (ไหน), (ต่อ), (คิว), (จ่าย), (เงิน), (คน), (ไหน), (รอ), (คน), (ไหน), (รอ), (กาแฟ), ]
[(สั่ง), (พวก), (เครื่อง), (ดื่ม), (เมนู), (ปั่น), (คือ), (เท), (ทุก), (อย่าง), (ลง), (ไป), (เลย), (ที), (เดียว), (ไม่), (ชง), (น้ำ), (ร้อน), (คน), (ให้), (เข้า), (กัน), (ก่อน), (เสีย), (สิ), (อี), (สาขา), (บ้าน), ]
[(สาขา), (มากๆ), (ๆๆๆ), 

[(สา), (ขา), (ทำไม), (เด็ก), (ใน), (ร้าน), (ถึง), (ออก), (ไม่), (เป็น), (เกือบ), (ทั้ง), (ร้าน), (ต้อง), (รอ), (คน), (เดียว), (ที่), (ทำ), (เป็น), ]
[(บ), (ปรุง), (เรื่อง), (มาตรฐาน), (ใน), (การ), (ชง), (หน่อย), (ค่ะ), (พนักงาน), (ชง), (กัน), (แต่ละ), (คน), (ไม่), (เหมือน), (กัน), (สัก), (คน), (เข้ม), (ไป), (หวาน), (ไป), (ไป), ]
[(เรื่อง), (ขนม), (มี), (เยอะ), (เลือก), (กิน), (ไม่), (ถูก), (ควร), (ปรุง), (ไม่), (งั้น), (เรา), (จะ), (อร่อย), (ไป), (แดก), ]
[(ชง), (อร่อย), (มาก), (ค่ะ), (พนักงาน), (บริการ), (ดี), (มาก), (โดย), (เฉพาะ), (สาขา), ]
[(ไป), (ซื้อ), (ตอน), (พน), (งบ), (อก), (ปิด), (ล้าง), (เครื่อง), (แล้ว), (คือ), (งง), (มาก), (เซ), (เว่น), (ต้อง), (เปิด), (ชม), (ไม่ใช่), (เห), (รอ), ]
[(พนัก), (งาน), (เซ), (เว่น), (คุย), (กัน), (มาก), (จริงๆ), (โดย), (เฉพาะ), (เวลา), (เปลี่ยน), (กะ), (คน), (เยอะ), (ยืน), (ออ), (กัน), (ใน), (เคา), (เตอร์), (คุย), (กัน), (ตั้ง), (แต่), (จน), (พนักงาน), (ช่วง), (เช้า), (ลูกค้า), (เยอะ), (เป็น), (ช่วง), (จะ), (เข้า), (พนง), (ร้าน), (ก็), (เยอะ), 

[(สรุป), (พนักงาน), (สุด), (ทุก), (อย่าง), (พวก), (ทำ), (แบบ), (เขา), (ได้), (ปะ), ]
[(ดี), (คุ้ม), (ค่า), (เกิน), (รา), (คา), (มากๆ), ]
[(ชา), (เย็น), (เข้ม), (แรง), (ดี), (คะ), (เคย), (ผสม), (กับ), (กาแฟ), (เย็น), (เลย), (คะ), ]
[(บาท), (ไป), (ดีด), (ยัน), (ตี), ]
[(หวาน), (ไป), ]
[(เปลี่ยน), (เป็น), (ชง), (สด), (จะ), (ดี), (มาก), (ค่ะ), (ปรับ), (หวาน), (มัน), (ได้), (ตาม), (ต้องการ), ]
[(บาง), (สาขา), (น้อง), (จะ), (ดู), (เวลา), (สั่ง), (เหมือน), (งาน), (เยอะ), (ไป), (ไม่), (น้อง), (นะ), ]
[(มอ), (ค), (ค่า), (อร่อย), ]
[(พนักงาน), (ทำ), (หน้า), (ส้นตีน), (ชอบ), (ของ), (คุย), (ไม่), (สนใจ), (ลูกค้า), ]
[(อย่า), (ทำ), (ไร), (ให้), (มัน), (มาก), (เกิน), ]
[(โกโก้), (ปั่น), (สี), (จืด), (เป็น), (จาง), (และ), (หวาน), (เจี๊ยบ), ]
[(อยาก), (ให้), (มี), (ผัก), (ไม่), (เหมือน), (แต่), (ก่อน), ]
[(อยาก), (ให้), (พนักงาน), (พูด), (ดี), (ไม่), (หน้า), (ก็), (พอ), (ค่ะ), (เลย), (ทุก), (สาขา), ]
[(ต้อง), (เอา), (ลา), (เต้), (ผสม), (โอ), (ตลอด), (เลย), (ใจ), (กุ), (สั่น), ]
[(อยาก), (ให้), (แก้ว),

[(เลิก), (ถาม), (ว่า), (ใส่), (ถุง), (ซัก), (ที), (เถอะ), (ครับ), ]
[(เครื่อง), (ชง), ]
[(บริการ), (และ), (รสชาติ), (ราคา), (การ), (เอา), (ของ), (บริการ), ]
[(คุย), (เวลา), (ชง), (เครื่อง), (ดื่ม), (น้ำลาย), (ใส่), (เท่า), (ไหร่), (ไม่รู้), ]
[(อร่อย), ]
[(อร่อย), (เป็น), (บาง), (สาขา), ]
[(แดก), (แก้ว), (เดียว), (ไทย), ]
[(ชง), (ให้), (มัน), (ละลาย), (ให้), (หมด), (ก่อน), (เท), (ให้), (เรา), (กิน), (มา), (เป็น), ]
[(เมนู), (อื่น), (ที่), (ไม่ใช่), (กาแฟ), (มัน), (ใช้), (ผง), (ชง), (อะ), (คือ), (ไม่), (อร่อย), (เลย), ]
[(ปรับปรุง), (ได้), (ของ), ]
[(คน), (ชง), (ค่า), (า), ]
[(เอา), (รถ), (มา), (ขาย), ]
[(ด้วย), (ราคา), (บาท), (ผม), (ละ), ]
[(ไม่), (ต้อง), (ปรับปรุง), (อะไร), (แล้ว), (ค่ะ), (เลิก), (ขาย), (ไป), (เลย), (ดี), (กว่า), ]
[(กาแฟ), (ที่), (นี่), (เขา), (จริง), (นะ), (เข้ม), (จน), (ใจ), (สั่น), ]
[(ลด), (นม), (หน่อย), (ใส่), (จน), ]
[(รสชาติ), (ก็), (โอ), (เค), (อยู่), (นะ), (คะ), (แต่), (หา), (คน), (ชง), (ไม่), (ค่อย), (มี), ]
[(หวาน), (ไป), (ค่ะ), ]
[(ชง), (เอง), (ที่), (บ้า

In [21]:
unique_words = []
for id in range(len(dictionary.keys())):
    unique_words.append(dictionary[id])

array = numpy.zeros((len_corpus, len(unique_words)), dtype=int)
for i, vector in enumerate(idx_corpus):
    for id in vector:
        array[i, id] += 1

word_vectors = pandas.DataFrame(array, columns=unique_words, dtype=int)
word_vectors

,กด,กาแฟ,กู,ของ,ครับ,ครึ่ง,ความ,จะ,ช่วย,ดีด,...,ไหร่,แบ,แย่,ป้า,เปน,ทาน,ครบ,คับ,โน,ขำ
0,1,2,1,1,3,1,2,1,1,2,...,0,0,0,0,0,0,0,0,0,4
1,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,5
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,8
6,0,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [57]:
cluster_num = len(clusters)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=cluster_num).fit(word_vectors)
predicted_labels = kmeans.labels_
print(numpy.unique(predicted_labels, return_counts=True))

(array([0, 1, 2, 3, 4], dtype=int32), array([380,  63, 482,   5, 161]))


In [58]:
for label in range(cluster_num):
    print('\t' + str(label), end='')
print('')

for cluster in clusters:
    print(cluster + '  |', end='')
    for label in range(cluster_num):
        count = 0
        for i in range(len_corpus):
            if predicted_labels[i] == label and labels[i] == cluster:
                #print('-', corpus[i].strip())
                count += 1
        print('\t' + str(count), end='')
    print('')
        #print('Cluster ' + cluster + ' have ' + str(count) + ' thread.\n')

	0	1	2	3	4
c  |	58	13	53	2	31
a  |	164	17	256	0	61
d  |	9	0	28	0	2
e  |	28	0	31	0	0
b  |	121	33	114	3	67
